<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#TF-Estimator-MNIST-Example" data-toc-modified-id="TF-Estimator-MNIST-Example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TF Estimator MNIST Example</a></span></li><li><span><a href="#TF-Slim-MNIST-Example" data-toc-modified-id="TF-Slim-MNIST-Example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TF Slim MNIST Example</a></span></li><li><span><a href="#TFLearn-MNIST-Example" data-toc-modified-id="TFLearn-MNIST-Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TFLearn MNIST Example</a></span></li><li><span><a href="#Pretty-Tensor-MNIST-Example" data-toc-modified-id="Pretty-Tensor-MNIST-Example-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pretty Tensor MNIST Example</a></span></li><li><span><a href="#Sonnet-MNIST-Example" data-toc-modified-id="Sonnet-MNIST-Example-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sonnet MNIST Example</a></span></li></ul></div>

# High-Level Libraries for TensorFlow <a class="tocSkip">

# TF Estimator MNIST Example 

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=False
                                  )
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

n_classes = 10
batch_size = 100
n_steps = 1000
learning_rate = 0.01


def model_fn(features, labels, mode):
    """ define the model function
    """
    espec_op = tf.estimator.EstimatorSpec
    # features is a dict as per Estimator specifications
    x = features['images']
    # define the network
    layer_1 = tf.layers.dense(x, 32)
    layer_2 = tf.layers.dense(layer_1, 32)
    logits = tf.layers.dense(layer_2, n_classes)

    # define predicted classes
    predicted_classes = tf.argmax(logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        espec = espec_op(mode,
                         predictions=predicted_classes
                         )
    else:
        # define loss and optimizer
        entropy_op = tf.nn.sparse_softmax_cross_entropy_with_logits
        loss_op = tf.reduce_mean(entropy_op(logits=logits,
                                            labels=tf.cast(labels,
                                                           dtype=tf.int32)
                                            )
                                 )
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss_op, global_step=tf.train.get_global_step())

        # define accuracy
        accuracy_op = tf.metrics.accuracy(
            labels=labels, predictions=predicted_classes)

        espec = espec_op(mode=mode,
                         predictions=predicted_classes,
                         loss=loss_op,
                         train_op=train_op,
                         eval_metric_ops={'accuracy': accuracy_op}
                         )

    return espec


# create estimator object
model = tf.estimator.Estimator(model_fn)

# train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)
model.train(train_input_fn, steps=n_steps)

# evaluate the model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test},
    y=y_test,
    batch_size=batch_size,
    shuffle=False)
model.evaluate(eval_input_fn)

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

# TF Slim MNIST Example

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import slim
import tensorflow as tf
import os

tf.reset_default_graph()

n_classes = 10  
n_steps = 1000

# let us get the data
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'), one_hot=True)

X_train = mnist.train.images
X_train = tf.convert_to_tensor(X_train)
Y_train = mnist.train.labels
Y_train = tf.convert_to_tensor(Y_train)


def mlp(x):
    net = slim.fully_connected(x, 32, scope='fc1')
    net = slim.dropout(net, 0.5, scope='dropout1')
    net = slim.fully_connected(net, 32, scope='fc2')
    net = slim.dropout(net, 0.5, scope='dropout2')
    net = slim.fully_connected(net, n_classes, activation_fn=None, scope='fc3')
    return net


# Define the model
logits = mlp(X_train)

# Define the loss functions and get the total loss
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y_train)
total_loss = tf.losses.get_total_loss()

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

train_op = slim.learning.create_train_op(total_loss, optimizer)

# Run the training
final_loss = slim.learning.train(
    train_op,
    logdir='./slim_logs',
    number_of_steps=n_steps,
    log_every_n_steps=100)

print('final loss={}'.format(final_loss))

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path ./slim_logs/model.ckpt
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global step 100: loss = 2.2669 (0.010 sec/step)
INFO:tensorflow:global step 200: loss = 2.2025 (0.010 sec/step)
INFO:tensorflow:global step 300: loss = 2.1257 (0.010 sec/step)
INFO:tensorflow:global step 400: loss = 2.0419 (0.009 sec/step)
INFO:tensorflow:global step 500: loss = 1.9532 (0.009 sec/step)
INFO:tensorflow:global step 600: loss = 1.8733 (0.010 sec/step)
INFO:tensorflow:global step 700: loss = 1.8002 (0.010 sec/step)
INFO:tensorflow:global step 800: loss = 1.7273 (0.010 sec/step)
INFO:tensorflow:global step 900: loss = 1.6688 (0.010 sec/st

# TFLearn MNIST Example

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

import tflearn
import tflearn.datasets.mnist as mnist
import os

batch_size = 100
n_classes = 10
n_epochs = 10

X_train, Y_train, X_test, Y_test = mnist.load_data(
    data_dir=os.path.join('.', 'mnist'), one_hot=True)

# Build deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
layer1 = tflearn.fully_connected(input_layer,
                                 10,
                                 activation='relu'
                                 )
layer2 = tflearn.fully_connected(layer1,
                                 10,
                                 activation='relu'
                                 )
output = tflearn.fully_connected(layer2,
                                 n_classes,
                                 activation='softmax'
                                 )

net = tflearn.regression(output,
                         optimizer='adam',
                         metric=tflearn.metrics.Accuracy(),
                         loss='categorical_crossentropy'
                         )
model = tflearn.DNN(net)

model.fit(
    X_train,
    Y_train,
    n_epoch=n_epochs,
    batch_size=batch_size,
    show_metric=True,
    run_id='dense_model')

score = model.evaluate(X_test, Y_test)
print('Test accuracy:', score[0])

# Pretty Tensor MNIST Example

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

import numpy as np
from __future__ import division

import prettytensor as pt
from prettytensor.tutorial import data_utils
import os
data_utils.WORK_DIRECTORY = os.path.join('.', 'mnist')

# get the data
X_train, Y_train = data_utils.mnist(training=True)
X_test, Y_test = data_utils.mnist(training=False)

# define hyperparameters

batch_size = 100     # number of samples that would
# be used to learn the parameters in a batch
n_classes = 10       # number of outputs, i.e. digits 0 to 9
n_epochs = 10        # number of ietrations for learning the parameters
n_batches = int(X_train.shape[0] / batch_size)
n_samples_in_train_batch = 60000 // batch_size
n_samples_in_test_batch = 10000 // batch_size

# define inputs and outputs

X = tf.placeholder(tf.float32, [batch_size, 28, 28, 1])
Y = tf.placeholder(tf.float32, [batch_size, 10])

# define the model

X = pt.wrap(X)

model = (X.
         flatten().
         fully_connected(10).
         softmax_classifier(n_classes, labels=Y)
         )

# define evaluator (metrics), optimizer and training functions

evaluator = model.softmax.evaluate_classifier(Y)
optimizer = tf.train.GradientDescentOptimizer(0.1)
trainer = pt.apply_optimizer(optimizer, losses=[model.loss])

runner = pt.train.Runner()

with tf.Session() as tfs:
    for epoch in range(0, n_epochs):
        # shuffle the training data
        X_train, Y_train = data_utils.permute_data((X_train, Y_train))

        runner.train_model(
            trainer,
            model.loss,
            n_samples_in_train_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_train, Y_train),
            print_every=600
        )

        score = runner.evaluate_model(
            evaluator,
            n_samples_in_test_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_test, Y_test)
        )

        print('Accuracy after {} epochs {} \n'.
              format(epoch + 1, score[0]))

# Sonnet MNIST Example

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

import os
import sonnet as snt

from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.INFO)


class MNIST(snt.AbstractModule):

    def __init__(self, mnist_part, batch_size, name='MNIST'):

        super(MNIST, self).__init__(name=name)

        self._X = tf.constant(mnist_part.images, dtype=tf.float32)
        self._Y = tf.constant(mnist_part.labels, dtype=tf.float32)
        self._batch_size = batch_size
        self._M = mnist_part.num_examples

    def _build(self):
        idx = tf.random_uniform([self._batch_size], 0, self._M, tf.int64)
        X = tf.gather(self._X, idx)
        Y = tf.gather(self._Y, idx)
        return X, Y


class MLP(snt.AbstractModule):
    def __init__(self, output_sizes, name='mlp'):
        super(MLP, self).__init__(name=name)

        self._layers = []

        for output_size in output_sizes:
            self._layers.append(snt.Linear(output_size=output_size))

    def _build(self, X):

        # add the input layer
        model = tf.sigmoid(self._layers[0](X))

        # add hidden layers
        for i in range(1, len(self._layers) - 1):
            model = tf.sigmoid(self._layers[i](model))

        # add output layer
        model = tf.nn.softmax(self._layers[len(self._layers) - 1](model))

        return model


batch_size = 100
n_classes = 10
n_epochs = 10

mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=True
                                  )
train = MNIST(mnist.train, batch_size=batch_size)
test = MNIST(mnist.test, batch_size=batch_size)

X_train, Y_train = train()
X_test, Y_test = test()

model = MLP([20, n_classes])

Y_train_hat = model(X_train)
Y_test_hat = model(X_test)


def loss(Y_hat, Y):
    return -tf.reduce_sum(Y * tf.log(Y_hat))


L_train = loss(Y_train_hat, Y_train)
L_test = loss(Y_test_hat, Y_test)

optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=0.01).minimize(L_train)

with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        loss_val, _ = tfs.run((L_train, optimizer))
        print('Epoch : {} Training Loss : {}'.format(epoch, loss_val))

    loss_val = tfs.run(L_test)
    print('Test loss : {}'.format(loss_val))